## HyperLang Sample Concept


In this sample concept, we will generate transcripts in Spanish given the country of origin and scenario. For now, we manually specify the country and scenario in this notebook, but, in the future we can create more
specific parameters (e.g. difficulty, number of speakers in conversation) that will be inputted by the end user.

### Helper Classes

In [1]:
"""
Handles LLM interactions using Hugging Face's API for text generation.
"""
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
from typing import Dict, Any, AsyncGenerator
import asyncio
import config
import os


load_dotenv()


class PromptHandler:
   """
   Class to handle loading and formatting of prompt templates.
   """
   
   def __init__(self, template_path: str) -> None:
       """Initialize with path to prompt template file."""
       self.template_path = template_path

   def load_prompt(self) -> str:
       """Load prompt template from file."""
       with open(self.template_path, 'r') as file:
           return file.read()

   def format_prompt(self, country_name: str, scenario: str) -> str:
       """
       Format prompt with country name and conversation scenario.
       
       Args:
           country_name: Country Name
           scenario: Conversation Scenario
           
       Returns:
           Formatted prompt for LLM
       """
       template = self.load_prompt()
       return template.format(
           country_name=country_name,
           scenario=scenario
       )

class LLMAdapter:
   """Handles LLM interactions for text generation."""
   
   def __init__(self, model_name: str = "mistralai/Mistral-7B-Instruct-v0.3", temperature: float = 0.7) -> None:
       """Initialize with model configuration."""
       self.client = InferenceClient(api_key=os.getenv('HUGGINGFACE_TOKEN'))
       self.model_name = model_name
       self.temperature = temperature

   def generate_response(self, prompt: str) -> str:
       """
       Generate complete response from LLM.
       
       Args:
           prompt: Input text for LLM
           
       Returns:
           Complete generated text
       """
       try:
           messages = [{"role": "user", "content": prompt}]
           
           completion = self.client.chat.completions.create(
               model=self.model_name,
               messages=messages,
               temperature=self.temperature,
               max_tokens=10000
           )
           
           return completion.choices[0].message.content
           
       except Exception as e:
           print(f"Exception occurred: {str(e)}")
           return f"Error: {str(e)}"

   async def generate_response_stream(self, prompt: str) -> AsyncGenerator[str, None]:
       """
       Generate streaming response from LLM.
       
       Args:
           prompt: Input text for LLM
           
       Returns:
           AsyncGenerator yielding text chunks
       """
       try:
           messages = [{"role": "user", "content": prompt}]
           
           stream = self.client.chat.completions.create(
               model=self.model_name,
               messages=messages,
               temperature=self.temperature,
               max_tokens=10000,
               stream=True
           )
           
           for chunk in stream:
               if chunk.choices[0].delta.content is not None:
                   yield chunk.choices[0].delta.content
                   await asyncio.sleep(0.01)  # small delay to control stream rate
               
       except Exception as e:
           print(f"Exception occurred in stream: {str(e)}")
           yield f"Error: {str(e)}"

/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/aryanmishra/Desktop/HyperLang/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
country_name = "Dominican Republic" ## specify the country
scenario  = "A tourist enters a café in Punta Cana and orders some local breakfast and coffee."  ## specify the scenario

In [3]:
prompt_handler = PromptHandler(config.PROMPT_TEMPLATE_PATH)
prompt = prompt_handler.format_prompt(country_name, scenario)

In [4]:
print(prompt)

You are an expert content creator for language learners in Spanish. Your task is to generate a realistic conversation in the form of a transcript between two characters given the following contextual parameters:

**Country where conversation takes place:**

Dominican Republic

**Specific scenario**

A tourist enters a café in Punta Cana and orders some local breakfast and coffee.

Ensure you generate a transcript that is clean and takes the dialect of the country in account. For example, when comparing Spanish spoken in Colombia versus Spanish spoken in Spain, 
there are certain differences between like the following: 

- If you want to say “You all are my best friends”, in Spain they would say “Vosotros sois mis mejores amigos” or “vosotras sois mis mejores amigas”. In Colombia they would say, “Ustedes son mis mejores amigos” 
or “ustedes son mis mejores amigas”. 
- If you want to say “Do you want to go out?”, in Spain they would say “¿Tenéis ganas de salir?” whereas in Colombia they 

In [5]:
llm_adapter = LLMAdapter(model_name=config.MODEL_NAME)

## Synchronous Output

In [6]:
response = llm_adapter.generate_response(prompt)
print(response)

Transcript:

Café en Punta Cana, Dominican Republic

(Tourist enters the café and looks at the menu)

Tourist (pointing at the menu): ¡Qué delicioso todo parece! ¿Por qué, ¿qué es esto? (What does this look like? What is this?)

Café Owner: Ah, ¡esto es un desayuno típico dominicano! Tiene mangu, queso frito, y una taza de café. (This is a typical Dominican breakfast! It has mangu, fried cheese, and a cup of coffee.)

Tourist: ¡Perfecto! ¿Puedo pedirlo, por favor? (Perfect! Can I order it, please?)

Café Owner: Claro, señor/señora. ¡Disfrútalo! (Of course, sir/madam. Enjoy it!)

(Tourist sits down and drinks their coffee)

Tourist: ¡Este café es delicioso! ¿Hay alguna cosa típica que recomiendes que trago con el desayuno? (This coffee is delicious! Is there something typical that you recommend I drink with the breakfast?)

Café Owner: ¡Claro! ¡El mamajuana es una bebida tradicional dominicana que es perfecta con el desayuno! (Of course! The mamajuana is a traditional Dominican drink th

## Asynchronous Output

In [7]:
async def stream_convo():
    async def convo_stream():
        async for token in llm_adapter.generate_response_stream(prompt):
            yield token
    async for token in convo_stream():
        print(token, end='', flush=True)

In [8]:
await stream_convo()

 **Transcript:**

Café Owner (CO): ¡Bienvenido! ¿Qué podemos preparar para desayunar hoy? (Welcome! What can we prepare for breakfast today?)

Tourist (T): Hola, quisiera pedir un desayuno típico y un café. (Hello, I'd like to order a typical breakfast and a coffee.)

CO: ¡Claro! Vamos a preparar un mangu, un tostón, un huevo frito y un café dominicano. (Of course! We'll prepare mangu, tostón, a fried egg, and a Dominican coffee for you.)

T: ¡Gracias! ¿Y el precio, por favor? (Thank you! And the price, please?)

CO: El precio es 200 pesos. (The price is 200 pesos.)

T: Entiendo. ¿También tienes algo dulce para la mesa? (I understand. Do you have something sweet for the table too?)

CO: ¡Sí! Tenemos medialunas y arroz con leche. (Yes, we have medialunas and rice pudding.)

T: ¡Muy bien! Yo quiero una medialuna y una taza de arroz con leche. (Great! I'd like a medialuna and a cup of rice pudding.)

CO: ¡Claro que sí! ¡Aquí está tu orden! (Of course! Here's your order!)

T: ¡Muchas graci